<div style="font-size: 200%; font-weight: bold; color: maroon;">Centralidad</div>
<!-- Paulo Villegas, 2020 - v.1.1 -->

In [ ]:
import networkx as nx

# Preparación

Creamos una función para pintar un grafo de NetworkX en el _notebook_ usando PyDot 

In [ ]:
from IPython.core.display import SVG, HTML
import re

def render(g, ptype="neato", **kwargs):
    '''
     :param g (Graph): the graph to render
     :param ptype (str): the GraphViz engine to use
     :param kwargs (dict): formatting options for nodes
    '''
    # Define default graph, edge & node properties
    g.graph['graph'] = {'nodesep': 3}
    #g.graph['graph'] = {'dpi': 160, 'size': '10', 'ratio': 'expand'}
    g.graph['edge'] = {'color': '#40e0d0d0'}
    
    node = {'style': 'filled', 'shape': 'circle'}
    node.update(kwargs)     # Add any passed node properties
    g.graph['node'] = node

    # Convert to GraphViz
    dot = nx.nx_pydot.to_pydot(g)

    # Render into SVG
    svg = dot.create_svg(prog=ptype)
    
    # Embed in an HTML div
    return HTML('<div style="">' + svg.decode() + '</div>')

    #svg = re.sub(b'(width|height)="[^"]+"', b'\\1="100%"', svg)
    #svg = re.sub(b'viewBox="([^"]+)"', b'viewBox="\\1" preserveAspectRatio="xMidYMid meet"', svg)
    #return svg
    #return SVG(svg)
    #return HTML('<div style="width: 860px; border: 1px solid red;">' + svg + '</div>')
    #return SVG(dot.create_svg(prog=ptype))

# Grafo

Usamos como ejemplo una **Red social** predefinda en NetworkX: la de familias florentinas

[Referencia NetworkX](https://networkx.github.io/documentation/latest/reference/generated/networkx.generators.social.florentine_families_graph.html)

In [ ]:
ff = nx.florentine_families_graph()
render(ff, shape='rectangle', fontsize=12, height=.3)

# Centralidad

NetworkX incorpora [muchas medidas de centralidad diferente](https://networkx.org/documentation/stable/reference/algorithms/centrality.html). Vamos a calcular algunas sobre este grafo

## Centralidad de grado

La centralidad de grado de un nodo es simplemente el grado de ese nodo, es decir el número de nodos vecinos. Se puede dividir por el número total de conexiones posible ($n-1$) para obtener la fracción de nodos conectados.

In [ ]:
dc = nx.degree_centrality(ff)

from operator import itemgetter
for f, c in sorted(dc.items(), key=itemgetter(1), reverse=True):
    print( '{:04.3f}'.format(c), f)

## Centralidad de intermediación

Esta centralidad (_betweenness centrality_) mide, para un nodo, la fracción total de caminos geodésicos que pasan por el nodo. Viene a representar en cuántas ocasiones el nodo es "de paso" entre dos rutas.

In [ ]:
bc = nx.betweenness_centrality(ff)

from operator import itemgetter
for f, c in sorted(bc.items(), key=itemgetter(1), reverse=True):
    print( '{:04.3f}'.format(c), f)

## Centralidad de autovector

Se calcula matemáticamente usando la matriz de adyacencia del grafo. En esta métrica la importancia de un nodo está relacionada con la importancia de sus vecinos


In [ ]:
ec = nx.eigenvector_centrality(ff)

from operator import itemgetter
for f, c in sorted(ec.items(), key=itemgetter(1), reverse=True):
    print( '{:04.3f}'.format(c), f)

# Comparativa

In [ ]:
import pandas as pd

data = {'family.degree': dc.keys(), 'value.degree': dc.values(),
        'family.between': bc.keys(), 'value.between': bc.values(),
        'family.eigen': ec.keys(), 'value.eigen': ec.values()}

df = pd.DataFrame(data)

In [ ]:
df

Ordenamos cada columna de centralidad

In [ ]:
for t in 'degree', 'between', 'eigen':
    cols = [f'family.{t}', f'value.{t}']
    df.loc[:,cols] = df[cols].sort_values(by=f'value.{t}', ascending=False).values

In [ ]:
df